## 建立迴歸分析變數

* 地點、旅遊類型、年份等資料虛擬變數

In [5]:
import pandas as pd
import numpy as np

original_review=pd.read_csv('original_review.csv')
original_review['is_Hotel']=original_review['hotel_rank'].apply(lambda x:1 if '飯店' in x else 0)
original_review['is_BnB']=original_review['hotel_rank'].apply(lambda x:1 if '民宿' in x else 0)
original_review['is_Specialty_lodging']=original_review['hotel_rank'].apply(lambda x:1 if '特色住宿' in x else 0)
ch_list=['信義', '萬華', '大同', '大安', '中山', '中正', '松山', '北投', '南港', '士林','內湖', '文山']
en_list=['Xinyi','Wanhua','Datong','Daan','Zhongshan','Zhongzheng','Songshan','Beitou','Nangang','Shilin','Neihu','Wenshan']
for ch,en in zip(ch_list,en_list):
    original_review[f'is_{en}']=original_review['hotel_rank'].apply(lambda x:1 if ch in x else 0)
original_review['year']=original_review['date'].apply(lambda x:x[:4] if pd.isna(x)==False else None)
for name in ['is_cp', 'is_place', 'is_service', 'is_comfort', 'is_clean', 'is_sleep']:
    original_review[name]=original_review[name]/10
    original_review=original_review.rename(columns={name:name[3:]+"_subrating"})
cn=['家庭旅行', '商務旅行', '單人旅行', '浪漫雙人遊', '好友出遊']
en=['family','business','single','double','friends']
for ch,en in zip(cn,en):
    original_review[f'Type_{en}']=original_review['trip'].apply(lambda x:0 if(( pd.isna(x))|(ch!=x))else 1 )
original_review['member_reviewsnum']=original_review['reviewsnum'].apply(lambda x: float(x.replace('篇投稿',"").replace(',','').strip()) if pd.isna(x)==False else None)
original_review['member_allhelpful']=original_review['allhelpful'].apply(lambda x:float(x.replace('人推薦',"").replace(',','').strip()) if pd.isna(x)==False else None)
original_review['hotel_total_rating']=original_review['hotel_total_rating']/10
original_review=original_review.rename(columns={'ReviewNum':'hotel_reviewsnum'}).drop(columns=['reviewsnum', 'member_name', 'place','allhelpful',
       'hotel_name','hotel_address', 'rating','hotel_rank',
       'hotel_subrate_place', 'hotel_subrate_clean', 'hotel_subrate_service',
       'hotel_subrate_cp', 'hotel_facility','title', 'review',
       'auto_translate','date', 'trip', 'total_dict'])

In [6]:
cut_sentence=pd.read_csv("Emotion_Score.csv")

In [17]:
for cate in ['service','place','clean','cp','comfort','sleep']:
    cate_features=original_review[list(original_review.columns[:-6]) +[f'{cate}_subrating']]
    for ye in [2019., 2020., 2018., 2017., 2016., 2015., 2014., 2013., 2012.,
       2011., 2010., 2009., 2007., 2008.]:
        cate_features[f'is_{int(ye)}']=cate_features['year'].apply(lambda x:1 if ye==x else 0)
    cate_features=cate_features.drop(columns='year')
    cate_features['hotel_star']=cate_features['hotel_star'].apply(lambda x:float(0 if x== 'no_hotel_star' else x))
    cut_sentence['negation_count']=cut_sentence[cut_sentence.has_negation.isnull()==False]['has_negation'].apply(len)
    cut_sentence['negation_count']=cut_sentence['negation_count'].fillna(0)
    by_review=cut_sentence[cut_sentence[f'word_in_{cate}'].isnull()==False]
    by_review_=by_review.groupby('old_index')[['emotion_score','positive_count','negative_count','weak_count','medium_count','strong_count','negation_count']].sum().reset_index()
    
    by_review_.merge(cate_features,how='left').to_csv(f'./Regression/Regression_{cate}.csv',index=False)

c:\users\jim\anaconda3\envs\nlp_tf\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
